<a href="https://colab.research.google.com/github/alexkociubinski/Monte-Carlo-Lattice-/blob/main/monteCarloLattice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import random

T = 2.3
size = 8
lattice = [[random.choice([1, -1]) for _ in range(size)] for _ in range(size)]

def calculate_energy(lattice, row, col):
    size = len(lattice)
    current_index = lattice[row][col]
    up = lattice[(row - 1) % size][col]
    down = lattice[(row + 1) % size][col]
    left = lattice[row][(col - 1) % size]
    right = lattice[row][(col + 1) % size]

    neighbors_sum = up + down + left + right

    energy = current_index * neighbors_sum
    return energy

steps = 2000000
for step in range(steps):
  row = random.randint(0, size -1)
  col = random.randint(0, size -1)

  previous_energy = calculate_energy(lattice, row, col)
  lattice[row][col] *= -1

  new_energy = calculate_energy(lattice, row, col)

  change_energy = new_energy - previous_energy


  if change_energy < 0:
        pass
  else:
    P = math.exp(-change_energy / T)
    if random.random() >= P:
        lattice[row][col] *= -1



for row in lattice:
    print(" ".join(f"{x:+2d}" for x in row))


+1 -1 +1 -1 +1 -1 +1 -1
-1 +1 -1 +1 -1 +1 -1 +1
+1 -1 +1 -1 +1 -1 -1 -1
-1 +1 -1 +1 -1 +1 -1 +1
+1 -1 +1 -1 +1 +1 +1 -1
-1 +1 -1 -1 -1 +1 -1 +1
+1 -1 +1 +1 +1 -1 +1 -1
-1 +1 -1 +1 -1 +1 -1 -1
